In [19]:
%load_ext autoreload
%autoreload 2

In [51]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [33]:
from utils import (
    make_applicants_df,
    plot_ability_distribution_by_sex
)
from unique_school import select_aa
from utility import calc_utility

### Parametars

In [4]:
female_a = 1959 # The number of female applicants
male_a = 7524 # The number of male applicants
q = 3000 # The number of students to be admitted

### Make applicants list

In [5]:
applicants_df = make_applicants_df(male_a, female_a, 0)

In [30]:
plot_ability_distribution_by_sex(applicants_df)

![Distribution](https://github.com/meina-t/matching_aa/blob/images/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88%202024-10-26%202.04.21.png)

### When there is no female quota

In [39]:
top_q_df, selected_df= select_aa(applicants_df, q, 0)

In [40]:
# The number of female who are admitted
len(selected_df[selected_df['sex'] == 'female'])

612

In [41]:
plot_ability_distribution_by_sex(selected_df)

In [42]:
ability_sum, diversity_factor, utility = calc_utility(selected_df)
print(f'ability_sum: {ability_sum}, diversity_factor: {diversity_factor}, utility: {utility}')

ability_sum: 182763.48994921026, diversity_factor: 0.162384, utility: 29677.866551912557


### When the quota changes

In [45]:
from run_unique_school import run_unique_school_with_quota

In [57]:
q_f_range = range(0, 1600, 100) # The range of the female quota
results_df = run_unique_school_with_quota(male_a, female_a, q, q_f_range)

In [50]:
results_df

,q_f,utility,ability_sum,diversity_factor
0,0,30044.461129,183307.798678,0.163890
1,100,30044.461129,183307.798678,0.163890
2,200,30044.461129,183307.798678,0.163890
3,300,30044.461129,183307.798678,0.163890
4,400,30044.461129,183307.798678,0.163890
5,500,30044.461129,183307.798678,0.163890
6,600,30105.652356,183307.717136,0.164224
7,700,32781.613772,183251.257113,0.178889
8,800,35793.554779,183035.223303,0.195556
9,900,38356.071027,182647.957271,0.210000


In [58]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for utility and ability_sum (left y-axis)
fig.add_trace(
    go.Scatter(x=results_df['q_f'], y=results_df['utility'], name="Utility", mode='lines'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=results_df['q_f'], y=results_df['ability_sum'], name="Ability Sum", mode='lines'),
    secondary_y=False
)

# Add trace for diversity_factor (right y-axis)
fig.add_trace(
    go.Scatter(x=results_df['q_f'], y=results_df['diversity_factor'], name="Diversity Factor", mode='lines', line=dict(color='black')),
    secondary_y=True
)

# Update layout
fig.update_layout(
    title="Utility, Ability Sum, and Diversity Factor vs. Q_f",
    xaxis_title="Q_f",
    yaxis_title="Utility / Ability Sum",
)

# Set secondary y-axis title for diversity_factor
fig.update_yaxes(title_text="Diversity Factor", secondary_y=True)

# Show plot
fig.show()

In [59]:
# Distribution when the quota for female is 1500
top_q_df, selected_df= select_aa(applicants_df, q, 1500)
plot_ability_distribution_by_sex(selected_df)

## When the number of female applicants changes

In [61]:
# Propotion of the female
female_a / (male_a + female_a)

0.20658019614046189

In [81]:
a_p_range = np.arange(0.05, 0.55, 0.05) 
a = male_a + female_a

In [82]:
a_p_range

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ])

In [83]:
from run_unique_school import run_unique_school_with_quota_female

In [84]:
results = run_unique_school_with_quota_female(a, q, a_p_range, q_f_range)

In [85]:
results

,female_proportion,q_f,utility,ability_sum,diversity_factor
0,0.05,0.0,8890.237051,183307.798678,0.048493
1,0.05,100.0,8890.237051,183307.798678,0.048493
2,0.05,200.0,11402.000304,183246.433455,0.062222
3,0.05,300.0,16440.406080,182671.178668,0.090000
4,0.05,400.0,20974.751941,181512.276415,0.115556
...,...,...,...,...,...
11,0.50,1100.0,45813.911183,183307.798678,0.249929
12,0.50,1200.0,45813.911183,183307.798678,0.249929
13,0.50,1300.0,45813.911183,183307.798678,0.249929
14,0.50,1400.0,45813.911183,183307.798678,0.249929


In [87]:
fig = px.line(results, x='q_f', y='utility', color='female_proportion', 
              title='Utility by female proportion and q_f',
              labels={'q_f': 'Quota for female', 'utility': 'utility', 'female_proportion': 'female_proportion'})
fig.show()